In [1]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_x = "i want to go shopping ."
data_y = "i will go to shop ."
data_x_split = data_x.split(' ')
data_y_split = data_y.split(' ')

data = data_x_split + data_y_split

data_list = ["<EOF>", "<PAD>"]
data_list.extend(list(set(data)))

print(data_list)

['<EOF>', '<PAD>', 'shopping', 'shop', 'i', '.', 'go', 'will', 'to', 'want']


In [3]:
class_num = len(data_list)
hidden_num = 4
learning_rate = 0.01
train_size = 20

In [4]:

data_x_idx = np.array([list_idx for x_idx, x_data in enumerate(data_x_split) for list_idx, list_data in enumerate(data_list) if list_data == x_data])
data_y_idx = np.array([list_idx for y_idx, y_data in enumerate(data_y_split) for list_idx, list_data in enumerate(data_list) if list_data == y_data])

data_x_idx = np.append(data_x_idx,0)
data_y_idx = np.append(data_y_idx,0)

ones_y_list = np.array(np.ones_like(data_x_idx[:len(data_x_idx)-1]))
data_y_idx = np.append(ones_y_list,data_y_idx)

data_x_idx = np.append(data_x_idx, np.ones(len(data_y_split), dtype=np.int64))


data_x_one_hot = np.zeros((len(data_x_idx), class_num))
data_y_one_hot = np.zeros((len(data_y_idx), class_num))

data_x_one_hot[np.arange(len(data_x_idx)), data_x_idx] = 1
data_y_one_hot[np.arange(len(data_y_idx)), data_y_idx] = 1

data_x_one_hot = np.array(data_x_one_hot, dtype='f')
data_y_one_hot = np.array(data_y_one_hot, dtype='f')

print(data_x_idx)
print(data_y_idx)

[4 9 8 6 2 5 0 1 1 1 1 1 1]
[1 1 1 1 1 1 4 7 6 8 3 5 0]


In [5]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, class_num])
y = tf.placeholder(tf.float32, [None, class_num])

input_w = tf.get_variable(name='input_w', shape=[class_num, hidden_num], initializer=tf.contrib.layers.xavier_initializer())
output_w = tf.get_variable(name="output_w", shape=[hidden_num, class_num], initializer=tf.contrib.layers.xavier_initializer())
prev_w = tf.get_variable(name="prev_w", shape=[class_num, hidden_num], initializer=tf.contrib.layers.xavier_initializer())
hidden_bias = tf.get_variable(name="hidden_bias", shape=[hidden_num], initializer=tf.contrib.layers.xavier_initializer())
output_bias = tf.get_variable(name="output_bias", shape=[class_num], initializer=tf.contrib.layers.xavier_initializer())
next_h = tf.get_variable(name="next_h", shape=[hidden_num], initializer=tf.zeros_initializer())

prev_h = tf.get_variable(name="prev_h", shape=[class_num], initializer=tf.zeros_initializer())
output = tf.get_variable(name="output", shape=(0))

def RnnCell(input_x, prev):
    input_data = tf.reshape(input_x, [-1, class_num])
    prev_h = tf.reshape(prev, [-1, class_num])
    hidden = tf.tanh(tf.matmul(input_data, input_w) + tf.matmul(prev_h, prev_w) + hidden_bias)
    output = tf.matmul(hidden, output_w) + output_bias
    return output

In [6]:
x_list = []
o_list = [prev_h]

for i in range(len(data_y_one_hot)):
    x_slice = tf.slice(x, [i,0], [1, -1])
    o_list.append(RnnCell(x_slice, o_list[i]))

output = o_list[1]

for i in range(len(o_list[2:])):
    output = tf.concat([output, o_list[i+2]], axis=0)
    
result = output

#data = output_y
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    prediction = tf.equal(tf.argmax(result,1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    
    for step in range(20):
        c = 0
        for i in range(train_size):
            _, c, pre_result = sess.run([optimizer, cost, tf.argmax(result, 1)], {x:data_x_one_hot, y:data_y_one_hot})

            if c<0.5:
                learning_rate = 0.001
        print("cost:",c)

    print("Accuracy:", accuracy.eval({x:data_x_one_hot, y:data_y_one_hot}))
    
    pre_result = sess.run(tf.argmax(result,1), {x:data_x_one_hot, y:data_y_one_hot})
    print(pre_result)
    print([data_list[i]for i in pre_result])

cost: 1.5747064
cost: 1.170388
cost: 0.92944324
cost: 0.70338506
cost: 0.5429994
cost: 0.4561842
cost: 0.39161715
cost: 0.34188908
cost: 0.30217668
cost: 0.27020997
cost: 0.24434
cost: 0.2231064
cost: 0.20540601
cost: 0.19040777
cost: 0.17748706
cost: 0.16618396
cost: 0.15616359
cost: 0.14718266
cost: 0.13906394
cost: 0.13167508
Accuracy: 1.0
[1 1 1 1 1 1 4 7 6 8 3 5 0]
['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', 'i', 'will', 'go', 'to', 'shop', '.', '<EOF>']
